# Predict general data

In [1]:
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path
import shutil
import csv
import re
from time import time
from collections import defaultdict
from datetime import datetime
import pandas as pd
import numpy as np
import torch

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append("./missing_data")

from dataset.datasets import MedicanesClsDataset
from torch.utils.data import DataLoader
import models
from timm.models import create_model

from mpl_toolkits.basemap import Basemap
from dataset.build_dataset import get_files_from_folder, extract_dates_pattern_airmass_rgb_20200101_0000
from dataset.build_dataset import load_cyclones_track_noheader, compute_pixel_scale, inside_tile, calc_tile_offsets, save_single_tile
from dataset.build_dataset import split_into_tiles_subfolders_and_track_dates, create_and_save_tile_from_complete_df
from dataset.build_dataset import create_final_df_csv
from dataset.build_dataset import get_gruppi_date, create_tile_videos, group_df_by_offsets
from medicane_utils.geo_const import latcorners, loncorners, x_center, y_center, create_basemap_obj
from medicane_utils.load_files import load_all_images

from view_test_tiles import create_labeled_images_with_tiles, create_mediterranean_video
from model_analysis import predict_label, get_path_pred_label, create_df_predictions, video_pred_2_img_pred

from arguments import prepare_finetuning_args, Args

from dataset.data_manager import BuildDataset, DataManager


file_master_df = "all_data_all_methods_tracks_complete_fast.csv"

output_dir = "../airmassRGB/supervised/" 


def sub_select_frequency(df, freq='20min'):
    # selezionare ore intere
    df['dt_floor'] = df['datetime'].dt.floor(freq)
    mask = df['datetime'] == df['dt_floor']
    df_filtered = df[mask]
    #grouped = df_filtered.groupby("path", dropna=False)
    return df_filtered

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/isac/miniconda3/envs/videomae/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Impossibile caricare la libreria torch dynamo


In [3]:
offsets = calc_tile_offsets(stride_x=113, stride_y=1313)
offsets

[(0, 0),
 (113, 0),
 (226, 0),
 (339, 0),
 (452, 0),
 (565, 0),
 (678, 0),
 (791, 0),
 (904, 0),
 (1017, 0)]

In [5]:
# non parto dal file di manos perché facciamo l'ipotesi di non avere label
data = BuildDataset(type='SUPERVISED', master_df_path=file_master_df)
data.load_master_df()

# non applico il relabeling

# filtro selezionando solo il 2023
print(data.master_df.shape[0])
df_2023 = data.master_df[data.master_df.datetime > datetime(2023,9,1)].copy()

data.make_df_video(new_master_df=df_2023, output_dir=output_dir) #,  is_to_balance=True)

csv_file = "./general_inference_set.csv"
data.create_final_df_csv(output_dir, csv_file)


1413168
1)  ->
Creazione delle folder per i 168 video...	Salvati 0 file - Erano già presenti 2688 file - File totali 2688
168 video per il periodo (effettivo) da 2023-09-03 00:00:00 a 2023-09-03 18:35:00

2)  ->
Creazione delle folder per i 1728 video...	Salvati 0 file - Erano già presenti 27648 file - File totali 27648
1728 video per il periodo (effettivo) da 2023-09-03 23:25:00 a 2023-09-11 23:30:00



In [6]:
#df_2023.drop(columns=['lat','lon','x_pix','y_pix','name'], inplace=True)
#df_2023

In [ ]:
# bonus -> mi guardo il video del periodo
#basemap_obj = create_basemap_obj()
grouped = df_2023.groupby("path", dropna=False)
print(len(grouped))
create_labeled_images_with_tiles(grouped, 'daniel_complete_tiles.gif', basemap_obj, 213, 196)

2545

# Inferenza

In [6]:
args = prepare_finetuning_args()
args.test_path = csv_file

test_m = DataManager(is_train=False, args=args, type_t='supervised', world_size=1, rank=0)
test_m.create_classif_dataloader(args)

Getting dataset...
DATASET length: 1896
Creo il DistributedSampler con world_size 1 e rank 0
Sampler_train = <torch.utils.data.distributed.DistributedSampler object at 0x7fbf01e51fd0>
Batch_size: 1


In [7]:
# carico il modello
get_prediction = True
args.test_mode = True
args.init_ckpt = './output/checkpoint-best-CL10.pth'

model = create_model(
    args.model,
    num_classes=args.nb_classes,
    drop_rate=0.0,
    drop_path_rate=args.drop_path,
    #attn_drop_rate=0.0,
    drop_block_rate=None,
    **args.__dict__
)

model.to(args.device)
model.eval()   


Caricamento del checkpoint da: ./output/checkpoint-best-CL10.pth
Caricato state_dict con chiave: model
Checkpoint caricato con successo!


VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv3d(3, 1408, kernel_size=(2, 14, 14), stride=(2, 14, 14))
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): ModuleList(
    (0): Block(
      (norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=1408, out_features=4224, bias=False)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=1408, out_features=1408, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=1408, out_features=6144, bias=True)
        (act): GELU(approximate=none)
        (fc2): Linear(in_features=6144, out_features=1408, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
  

In [8]:
# ottengo il dataframe delle predizioni

all_paths, all_preds, all_labels = get_path_pred_label(model, test_m.data_loader)
df_predictions = create_df_predictions(all_paths, all_preds, all_labels)

In [9]:
df_video_w_predictions = data.df_video.merge(df_predictions, on='path')

In [10]:
df_video_w_predictions#.columns

,video_id,tile_offset_x,tile_offset_y,path,label,start_time,end_time,orig_paths,predictions,labels
0,0,0,0,03-09-2023_0115_0_0,0,2023-09-03 00:00:00,2023-09-03 01:15:00,"[../fromgcloud/airmass_rgb_20230903_0000.png, ...",0,0
1,1,0,0,03-09-2023_0235_0_0,0,2023-09-03 01:20:00,2023-09-03 02:35:00,"[../fromgcloud/airmass_rgb_20230903_0120.png, ...",0,0
2,2,0,0,03-09-2023_0355_0_0,0,2023-09-03 02:40:00,2023-09-03 03:55:00,"[../fromgcloud/airmass_rgb_20230903_0240.png, ...",0,0
3,3,0,0,03-09-2023_0515_0_0,0,2023-09-03 04:00:00,2023-09-03 05:15:00,"[../fromgcloud/airmass_rgb_20230903_0400.png, ...",0,0
4,4,0,0,03-09-2023_0635_0_0,0,2023-09-03 05:20:00,2023-09-03 06:35:00,"[../fromgcloud/airmass_rgb_20230903_0520.png, ...",0,0
...,...,...,...,...,...,...,...,...,...,...
1891,1723,1065,196,11-09-2023_1810_1065_196,0,2023-09-11 16:55:00,2023-09-11 18:10:00,"[../fromgcloud/airmass_rgb_20230911_1655.png, ...",0,0
1892,1724,1065,196,11-09-2023_1930_1065_196,0,2023-09-11 18:15:00,2023-09-11 19:30:00,"[../fromgcloud/airmass_rgb_20230911_1815.png, ...",0,0
1893,1725,1065,196,11-09-2023_2050_1065_196,0,2023-09-11 19:35:00,2023-09-11 20:50:00,"[../fromgcloud/airmass_rgb_20230911_1935.png, ...",0,0
1894,1726,1065,196,11-09-2023_2210_1065_196,0,2023-09-11 20:55:00,2023-09-11 22:10:00,"[../fromgcloud/airmass_rgb_20230911_2055.png, ...",0,0


In [11]:

df_mapping = video_pred_2_img_pred(df_video_w_predictions)

In [12]:
df_mapping

,path,predictions,tmp_label,tile_offset_x,tile_offset_y
0,../fromgcloud/airmass_rgb_20230903_0000.png,0,0,0,0
1,../fromgcloud/airmass_rgb_20230903_0005.png,0,0,0,0
2,../fromgcloud/airmass_rgb_20230903_0010.png,0,0,0,0
3,../fromgcloud/airmass_rgb_20230903_0015.png,0,0,0,0
4,../fromgcloud/airmass_rgb_20230903_0020.png,0,0,0,0
...,...,...,...,...,...
30331,../fromgcloud/airmass_rgb_20230911_2310.png,0,0,1065,196
30332,../fromgcloud/airmass_rgb_20230911_2315.png,0,0,1065,196
30333,../fromgcloud/airmass_rgb_20230911_2320.png,0,0,1065,196
30334,../fromgcloud/airmass_rgb_20230911_2325.png,0,0,1065,196


In [13]:
# se voglio limitare i frame da renderizzare

df_2023_20m = sub_select_frequency(df_2023)
print(df_2023_20m.shape[0])

7644


In [14]:
# ottengo il dataframe per creare i video
df_data_merg = df_mapping.merge(df_2023_20m, on=['path', 'tile_offset_x', 'tile_offset_y'], how='left').drop(columns='label').rename(columns={'tmp_label':'label'})

In [24]:
offsets = list(df_data_merg[['tile_offset_x','tile_offset_y']].value_counts().index.values)
offsets

[(0, 0),
 (0, 196),
 (213, 0),
 (213, 196),
 (426, 0),
 (426, 196),
 (639, 0),
 (639, 196),
 (852, 0),
 (852, 196),
 (1065, 0),
 (1065, 196)]

In [21]:
df_data_merg

,path,predictions,label,tile_offset_x,tile_offset_y,datetime,x_pix,y_pix,source,id_cyc_unico,start_time,end_time,dt_floor
0,../fromgcloud/airmass_rgb_20230903_0000.png,0,0,0,0,2023-09-03 00:00:00,[],[],[],0.0,NaT,NaT,2023-09-03 00:00:00
1,../fromgcloud/airmass_rgb_20230903_0005.png,0,0,0,0,NaT,NaN,NaN,<NA>,NaN,NaT,NaT,NaT
2,../fromgcloud/airmass_rgb_20230903_0010.png,0,0,0,0,NaT,NaN,NaN,<NA>,NaN,NaT,NaT,NaT
3,../fromgcloud/airmass_rgb_20230903_0015.png,0,0,0,0,NaT,NaN,NaN,<NA>,NaN,NaT,NaT,NaT
4,../fromgcloud/airmass_rgb_20230903_0020.png,0,0,0,0,2023-09-03 00:20:00,[],[],[],0.0,NaT,NaT,2023-09-03 00:20:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30331,../fromgcloud/airmass_rgb_20230911_2310.png,0,0,1065,196,NaT,NaN,NaN,<NA>,NaN,NaT,NaT,NaT
30332,../fromgcloud/airmass_rgb_20230911_2315.png,0,0,1065,196,NaT,NaN,NaN,<NA>,NaN,NaT,NaT,NaT
30333,../fromgcloud/airmass_rgb_20230911_2320.png,0,0,1065,196,2023-09-11 23:20:00,[],[],[],0.0,NaT,NaT,2023-09-11 23:20:00
30334,../fromgcloud/airmass_rgb_20230911_2325.png,0,0,1065,196,NaT,NaN,NaN,<NA>,NaN,NaT,NaT,NaT


In [25]:
from view_test_tiles import get_writer4animation, make_animation
os.environ['PATH'] = './ffmpeg-7.0.2-amd64-static:' + os.environ['PATH']

In [30]:
make_animation(df_data_merg, nomefile='daniel_prediction_1.mp4', writer=get_writer4animation())

 abbiamo 2528 gruppi


12.14 minuti
Video salvato: daniel_prediction_1.mp4


# old

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 500)

In [ ]:
master_id_df = pd.read_csv('master_data_2020_wID.csv')
master_df = pd.read_csv('all_data_full_tiles.csv').drop(columns=['Unnamed: 0'])

In [ ]:
#master_id_df.sort_values(by='datetime')


In [ ]:
#master_df.sort_values(by='datetime')

In [ ]:
master_id_df.path.isin(master_df.path).sum()

279372

In [ ]:
master_df[cols]

,path,datetime,tile_offset_x,tile_offset_y,label
0,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01 00:00:00,0,0,0
1,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01 00:00:00,213,0,0
2,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01 00:00:00,426,0,0
3,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01 00:00:00,639,0,0
4,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01 00:00:00,852,0,0
...,...,...,...,...,...
1413163,../fromgcloud/airmass_rgb_20230912_0000.png,2023-09-12 00:00:00,213,196,0
1413164,../fromgcloud/airmass_rgb_20230912_0000.png,2023-09-12 00:00:00,426,196,0
1413165,../fromgcloud/airmass_rgb_20230912_0000.png,2023-09-12 00:00:00,639,196,0
1413166,../fromgcloud/airmass_rgb_20230912_0000.png,2023-09-12 00:00:00,852,196,0


In [ ]:
master_id_df[cols]

,path,datetime,tile_offset_x,tile_offset_y,label
0,../fromgcloud/airmass_rgb_20200101_1200.png,2020-01-01 12:00:00,0,0,0
1,../fromgcloud/airmass_rgb_20200101_1200.png,2020-01-01 12:00:00,213,0,0
2,../fromgcloud/airmass_rgb_20200101_1200.png,2020-01-01 12:00:00,426,0,0
3,../fromgcloud/airmass_rgb_20200101_1200.png,2020-01-01 12:00:00,639,0,0
4,../fromgcloud/airmass_rgb_20200101_1200.png,2020-01-01 12:00:00,852,0,0
...,...,...,...,...,...
279367,../fromgcloud/airmass_rgb_20201228_0100.png,2020-12-28 01:00:00,213,196,0
279368,../fromgcloud/airmass_rgb_20201228_0100.png,2020-12-28 01:00:00,426,196,0
279369,../fromgcloud/airmass_rgb_20201228_0100.png,2020-12-28 01:00:00,639,196,0
279370,../fromgcloud/airmass_rgb_20201228_0100.png,2020-12-28 01:00:00,852,196,0


In [ ]:
cols = ['path','datetime','tile_offset_x','tile_offset_y','label']
res = pd.merge(master_df[cols], master_id_df[cols], on=cols, how='inner')

In [ ]:
master_id_df[master_id_df.path.str.contains('airmass_rgb_20200101_1200')][cols]

,path,datetime,tile_offset_x,tile_offset_y,label
0,../fromgcloud/airmass_rgb_20200101_1200.png,2020-01-01 12:00:00,0,0,0
1,../fromgcloud/airmass_rgb_20200101_1200.png,2020-01-01 12:00:00,213,0,0
2,../fromgcloud/airmass_rgb_20200101_1200.png,2020-01-01 12:00:00,426,0,0
3,../fromgcloud/airmass_rgb_20200101_1200.png,2020-01-01 12:00:00,639,0,0
4,../fromgcloud/airmass_rgb_20200101_1200.png,2020-01-01 12:00:00,852,0,0
5,../fromgcloud/airmass_rgb_20200101_1200.png,2020-01-01 12:00:00,1065,0,0
6,../fromgcloud/airmass_rgb_20200101_1200.png,2020-01-01 12:00:00,0,196,0
7,../fromgcloud/airmass_rgb_20200101_1200.png,2020-01-01 12:00:00,213,196,0
8,../fromgcloud/airmass_rgb_20200101_1200.png,2020-01-01 12:00:00,426,196,0
9,../fromgcloud/airmass_rgb_20200101_1200.png,2020-01-01 12:00:00,639,196,0


In [ ]:
master_df[master_df.path.str.contains('airmass_rgb_20200101_1200')][cols]

,path,datetime,tile_offset_x,tile_offset_y,label
153564,../fromgcloud/airmass_rgb_20200101_1200.png,2020-01-01 12:00:00,0,0,0
153565,../fromgcloud/airmass_rgb_20200101_1200.png,2020-01-01 12:00:00,213,0,0
153566,../fromgcloud/airmass_rgb_20200101_1200.png,2020-01-01 12:00:00,426,0,0
153567,../fromgcloud/airmass_rgb_20200101_1200.png,2020-01-01 12:00:00,639,0,0
153568,../fromgcloud/airmass_rgb_20200101_1200.png,2020-01-01 12:00:00,852,0,0
153569,../fromgcloud/airmass_rgb_20200101_1200.png,2020-01-01 12:00:00,1065,0,0
153570,../fromgcloud/airmass_rgb_20200101_1200.png,2020-01-01 12:00:00,0,196,0
153571,../fromgcloud/airmass_rgb_20200101_1200.png,2020-01-01 12:00:00,213,196,0
153572,../fromgcloud/airmass_rgb_20200101_1200.png,2020-01-01 12:00:00,426,196,0
153573,../fromgcloud/airmass_rgb_20200101_1200.png,2020-01-01 12:00:00,639,196,0


In [ ]:
res[res.path.str.contains('airmass_rgb_20200101_1200')]

,path,datetime,tile_offset_x,tile_offset_y,label
0,../fromgcloud/airmass_rgb_20200101_1200.png,2020-01-01 12:00:00,0,0,0
1,../fromgcloud/airmass_rgb_20200101_1200.png,2020-01-01 12:00:00,213,0,0
2,../fromgcloud/airmass_rgb_20200101_1200.png,2020-01-01 12:00:00,426,0,0
3,../fromgcloud/airmass_rgb_20200101_1200.png,2020-01-01 12:00:00,639,0,0
4,../fromgcloud/airmass_rgb_20200101_1200.png,2020-01-01 12:00:00,852,0,0
5,../fromgcloud/airmass_rgb_20200101_1200.png,2020-01-01 12:00:00,1065,0,0
6,../fromgcloud/airmass_rgb_20200101_1200.png,2020-01-01 12:00:00,0,196,0
7,../fromgcloud/airmass_rgb_20200101_1200.png,2020-01-01 12:00:00,213,196,0
8,../fromgcloud/airmass_rgb_20200101_1200.png,2020-01-01 12:00:00,426,196,0
9,../fromgcloud/airmass_rgb_20200101_1200.png,2020-01-01 12:00:00,639,196,0


In [ ]:
all(res == master_id_df[cols])  # -> allora il nuovo master df è completamente contenuto nel vecchio

True